**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

#### Execute the below cell to load the datasets

In [107]:
import pandas as pd
import os
import numpy as np
from surprise.dataset import Dataset,DatasetAutoFolds
from surprise import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD

In [108]:
os.chdir("C:\\Users\\tshrgpt329\AIML PG")

#Loading data
books = pd.read_csv("books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

users = pd.read_csv('users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

ratings = pd.read_csv('ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\IPython\core\interactivesh

### Check no.of records and features given in each dataset

In [109]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
ISBN                 271360 non-null object
bookTitle            271360 non-null object
bookAuthor           271359 non-null object
yearOfPublication    271360 non-null object
publisher            271358 non-null object
imageUrlS            271360 non-null object
imageUrlM            271360 non-null object
imageUrlL            271357 non-null object
dtypes: object(8)
memory usage: 16.6+ MB


In [110]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
userID      278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [111]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
userID        1149780 non-null int64
ISBN          1149780 non-null object
bookRating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


## Exploring books dataset

In [112]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop last three columns containing image URLs which will not be required for analysis

In [113]:
books.drop(columns=['imageUrlS','imageUrlM','imageUrlL'],inplace = True , axis=1)

In [114]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


**yearOfPublication**

### Check unique values of yearOfPublication


In [115]:
books['yearOfPublication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [116]:
books[books['yearOfPublication']=='DK Publishing Inc']

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [117]:
list_of_bad_rows_index = list(books[books['yearOfPublication']=='DK Publishing Inc'].index)+list(books[books['yearOfPublication']=='Gallimard'].index)
books.drop(index=list_of_bad_rows_index,axis=0,inplace=True)

Let's verify

In [118]:
books[books['yearOfPublication']=='DK Publishing Inc']

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


In [119]:
books[books['yearOfPublication']=='Gallimard']

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


### Change the datatype of yearOfPublication to 'int'

In [120]:
books=books.astype({'yearOfPublication':int})

In [121]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication     int32
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [122]:
books[books['publisher'].isna()]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN


In [123]:
books.dropna(axis=0,subset=['publisher'],inplace=True)

Let's verify

In [124]:
books[books['publisher'].isna()]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


## Exploring Users dataset

In [125]:
print(users.shape)
users.head()

(278858, 3)


,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### Get all unique values in ascending order for column `Age`

In [126]:
np.sort(users['Age'].unique())

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 113., 114., 115., 116., 118., 119., 123., 124., 127.,
       128., 132., 133., 136., 137., 138., 140., 141., 143., 146., 147.,
       148., 151., 152., 156., 157., 159., 162., 168., 172., 175., 183.,
       186., 189., 199., 200., 201., 204., 207., 20

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

Let's count how many rows are there

In [127]:
users['Age'][users['Age']<5].count()+users['Age'][users['Age']>99].count()

1255

In [128]:
users.mask(users['Age']<5,inplace=True)
users.mask(users['Age']>99,inplace=True)

Let's verify

In [129]:
users['Age'][users['Age']<5].count()+users['Age'][users['Age']>99].count()

0

In [130]:
np.sort(users['Age'].unique())

array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
       18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.,
       31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43.,
       44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56.,
       57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69.,
       70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82.,
       83., 84., 85., 86., 87., 88., 89., 90., 91., 92., 93., 94., 95.,
       96., 97., 98., 99., nan])

### Replace null values in column `Age` with mean

In [131]:
users['Age'].fillna(value=round(users['Age'].mean()),inplace=True)

Let's verify

In [132]:
np.sort(users['Age'].unique())

array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
       18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.,
       31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43.,
       44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56.,
       57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69.,
       70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82.,
       83., 84., 85., 86., 87., 88., 89., 90., 91., 92., 93., 94., 95.,
       96., 97., 98., 99.])

### Change the datatype of `Age` to `int`

In [133]:
users=users.astype({'Age':int})

In [134]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


## Exploring the Ratings Dataset

### check the shape

In [135]:
ratings.shape

(1149780, 3)

In [136]:
n_users = users.shape[0]
n_books = books.shape[0]

In [137]:
ratings.head(5)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [138]:
new_r=ratings.merge(right=books,how='inner',on='ISBN')
new_r.drop(columns=books.columns[1:],inplace=True,axis=1)
ratings = new_r
ratings.shape

(1031130, 3)

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [139]:
new_r=ratings.merge(right=users,how='inner',on='userID')
new_r.drop(columns=users.columns[1:],inplace=True,axis=1)
ratings = new_r
ratings.shape

(1026380, 3)

In [140]:
ratings.head()

,userID,ISBN,bookRating
0,276725,034545104X,0
1,2313,034545104X,5
2,2313,0812533550,9
3,2313,0679745580,8
4,2313,0060173289,9


### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [141]:
ratings.drop(index=list(ratings[ratings['bookRating']==0].index),axis=0,inplace=True)

let's verify

In [142]:
ratings[ratings['bookRating']==0]

,userID,ISBN,bookRating


In [143]:
ratings.head()

,userID,ISBN,bookRating
1,2313,034545104X,5
2,2313,0812533550,9
3,2313,0679745580,8
4,2313,0060173289,9
5,2313,0385482388,5


### Find out which rating has been given highest number of times

In [144]:
ratings['bookRating'].mode()

0    8
dtype: int64

### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [145]:
g = ratings.groupby(by='userID')
new_grp={}
for each_key in g.indices.keys():
    if(len(g.indices[each_key])>=100):
        new_grp[each_key]=g.indices[each_key]
        

In [146]:
new_ratings = pd.DataFrame()
for each_key in new_grp.keys():
    new_ratings=new_ratings.append(ratings.iloc[new_grp[each_key]])

In [147]:
new_ratings.count()

userID        102975
ISBN          102975
bookRating    102975
dtype: int64

### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [148]:
new_ratings[new_ratings.bookRating.isna()]

,userID,ISBN,bookRating


### Generate the predicted ratings using SVD with no.of singular values to be 50

In [149]:
reader = Reader(rating_scale=(new_ratings['bookRating'].min(), new_ratings['bookRating'].max()))
trainset = Dataset.load_from_df(new_ratings,reader)

In [150]:
algo = SVD()
algo.fit(trainset.build_full_trainset())

### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [151]:
userID = 2110

In [152]:
user_id = 2 #2nd row in ratings matrix and predicted matrix

### Get the predicted ratings for userID `2110` and sort them in descending order

In [216]:
d = []
for each_isbn in books['ISBN']:
    d.append([each_isbn,round(algo.predict(userID,each_isbn).est)]) 

In [217]:
predicted_df = pd.DataFrame(columns=['ISBN','Predicted_rating'],data=d)
predicted_df.head()

,ISBN,Predicted_rating
0,0195153448,8.0
1,0002005018,8.0
2,0060973129,8.0
3,0374157065,8.0
4,0393045218,8.0


Since we have captured the 'ISBN' and predicted ratings for the given user in a dataframe, it can be sorted as and when required.

In [218]:
predicted_df[predicted_df['Predicted_rating'].isna()]

,ISBN,Predicted_rating


### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [156]:
user_data=trainset.df[trainset.df['userID']==2110]

In [157]:
user_data.head()

,userID,ISBN,bookRating
667785,2110,059035342X,10
667789,2110,0590448595,8
667792,2110,0451137965,9
667793,2110,0590629786,10
667794,2110,0590629794,10


In [158]:
user_data.shape

(103, 3)

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [159]:
user_full_info = user_data.merge(on='ISBN',right=books)

In [160]:
book_data=user_full_info.drop(columns=['userID','bookRating'],axis=1)

In [161]:
book_data.shape

(103, 5)

In [162]:
book_data.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
1,0590448595,Karen's School Trip (Baby-Sitters Little Siste...,Ann M. Martin,1992,Scholastic Paperbacks (Mm)
2,0451137965,Thinner,Stephen King,1985,New Amer Library
3,0590629786,"The Visitor (Animorphs, No 2)",K. A. Applegate,1996,Scholastic
4,0590629794,"The Encounter (Animorphs , No 3)",K. A. Applegate,1996,Scholastic


In [163]:
user_full_info.head()

,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
0,2110,059035342X,10,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
1,2110,0590448595,8,Karen's School Trip (Baby-Sitters Little Siste...,Ann M. Martin,1992,Scholastic Paperbacks (Mm)
2,2110,0451137965,9,Thinner,Stephen King,1985,New Amer Library
3,2110,0590629786,10,"The Visitor (Animorphs, No 2)",K. A. Applegate,1996,Scholastic
4,2110,0590629794,10,"The Encounter (Animorphs , No 3)",K. A. Applegate,1996,Scholastic


### Get top 10 recommendations for above given userID from the books not already rated by that user

In [178]:
books_not_rated = pd.concat([books,book_data]).drop_duplicates(keep=False)

In [226]:
books_not_rated=books_not_rated.merge(right=predicted_df,on='ISBN')

In [230]:
books_not_rated.nlargest(10,columns='Predicted_rating')

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,Predicted_rating
37,0446310786,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,10.0
1383,0345361792,A Prayer for Owen Meany,John Irving,1990,Ballantine Books,10.0
2225,0812550706,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1994,Tor Books,10.0
5017,0060959037,Prodigal Summer: A Novel,Barbara Kingsolver,2001,Perennial,10.0
5458,043935806X,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,10.0
5850,0553274295,Where the Red Fern Grows,Wilson Rawls,1984,Random House Children's Books,10.0
6282,0439136369,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic,10.0
8733,0140143505,84 Charing Cross Road,Helene Hanff,1990,Penguin Books,10.0
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,9.0
18,0440234743,The Testament,John Grisham,1999,Dell,9.0


Above is the list of top 10 books predicted by reccomendation system.